# 1. Basic code
**Always needs to be executed even if some steps are skipped!**

The results of most steps are stored when the step is finished. Each step starts with reading the result of the previous step. This allows skipping steps, if the previous steps were already executed sometime else.

The main steps are:
1. Preprocessing (create spectrograms and splits)
2. Train the models
3. Evaluate the models
4. Run XAI methods on the models (see other script?)
5. Create Plots of waveform/spectrograms

## Mount Google Drive

In [ ]:
from google.colab import drive #mount Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


## Imports

In [ ]:
# for importing from py scripts in the root directory (including subfolders)
import sys

# for function get_duration
import math
import time

# for function write_log
import logging

# for preprocessing Samek
import numpy as np
import glob
import os
import sys
import scipy.io.wavfile as wavf
import scipy.signal
import h5py
import json
import librosa
import multiprocessing
#import argparse # dont need this one

# for preprocessing adjustments
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
# import shutil

# for reading hdf5 files
import h5py

# for model training
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import pickle
import tensorflow as tf


# for model evaluation
from tensorflow.keras.models import Model, load_model
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt

# for xai methods




#########
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# import warnings

# from tensorflow.keras.applications import VGG16 #Xception #ResNet50, VGG16
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras import layers
# from tensorflow.keras import optimizers
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from tensorflow.keras.callbacks import ModelCheckpoint

# import tensorflow as tf

# from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
# from sklearn.model_selection import KFold, train_test_split
# import pickle



## Set paths and import py scripts

In [ ]:
# set root dir: the path to the github repo folder "XAI_spec_TSC" in your google drive
root_dir = '/gdrive/My Drive/XAI_spec_AudioMNIST/'

paths = {
    'root': root_dir,
    'dataset': os.path.join(root_dir,'AudioMNIST-master'),
    'data': os.path.join(root_dir,'AudioMNIST-master/data'),
    'meta': os.path.join(root_dir,'AudioMNIST-master/data/audioMNIST_meta.txt'),
    'spectrograms': os.path.join(root_dir,'spectrograms'),
    'splits': os.path.join(root_dir,'splits'),
    'results': os.path.join(root_dir, 'results'),
    'models': os.path.join(root_dir, 'results/models'),
    'history': os.path.join(root_dir, 'results/history'),
    'evaluation': os.path.join(root_dir, 'results/evaluation'),
    'xai': os.path.join(root_dir, 'results/xai'),
    'plots': os.path.join(root_dir, 'results/plots'),
    'plots_waveform': os.path.join(root_dir, 'results/plots/waveform'),
    'plots_spectrograms': os.path.join(root_dir, 'results/plots/spectrograms')
}

# labels and number of splits
labels = ['gender','digit']
splits = 5

# append the directory to the python path using sys in order to make the seperate py files importable
sys.path.append(root_dir)

## Utils
Contains useful functions, which are used frequently during the code / in severeal sections

In [ ]:
########## 1. function to write log about the events
def write_log(message):
    logging.basicConfig(level = logging.INFO, filename=os.path.join(root_dir,'events.log'), filemode='a', format='%(asctime)s - %(message)s')
    logging.info(message)
    print(message) 


########## 2. function to create a directory
def create_directory(directory_path):
    if os.path.isdir(directory_path) == False:
        os.mkdir(directory_path)
        write_log('Created folder: '+directory_path)
    #else:
        #writeLog('Folder already exists: '+directory_path)

########## 3. function to calculate time difference between two timepoints from package 'time' and returns the duration in format HH:MM:SS as String
def get_duration(start_time,end_time):
    duration = round(end_time-start_time)
    if duration < 0:
        duration*=-1
    h=math.floor(duration/3600)
    r=duration%3600
    m=math.floor(r/60)
    r=r%60
    s=round(r)
    return(str(h).zfill(2)+':'+str(m).zfill(2)+':'+str(s).zfill(2))

########### 4. function to read spectrograms/labels and return np.arrays ready for training/evaluation/xai methods
def read_data(label,split_index,split_type):
  write_log('Started reading '+str(split_type)+' data ...')
  start_time = time.time()
  if label == 'gender':
    label_index = 1
  else:
    label_index = 0
  # read txt with current split paths
  path_to_split_paths = os.path.join(paths['splits'],'AlexNet_'+str(label)+'_'+str(split_index)+'_'+str(split_type)+'.txt')
  text_file = open(path_to_split_paths, 'r')
  split_paths = text_file.read().split('\n')
  text_file.close()
  # if there are empty lines at the end of the txt file there will be an empty list element for each empty line
  # removing empty lines/list elements
  while split_paths[len(split_paths)-1] == '':
    split_paths.pop(len(split_paths)-1)
  # read hdf5 files of the current split and split_type and store it as np.array (spectrograms as x and labels as y)
  index = 0
  x = np.zeros(((len(split_paths),227,227))) # create target array for spectrograms
  y = np.zeros(len(split_paths)) # create target array for labels
  for cur_path in split_paths: # iterate the files
    #read current file
    f = h5py.File(cur_path, 'r')
    x_cur = f['data'][...]
    y_cur = f['label'][...]
    f.close() 
    #extract relevant data of current file
    x_cur = x_cur[0][0]
    y_cur = y_cur[0][label_index]    
    #append current data to x and y
    x[index] = x_cur
    y[index] = y_cur
    # increase index by 1
    index +=1
  write_log('Finished reading '+str(split_type)+' data in '+get_duration(start_time,time.time()))
  return x,y

#create directories for the results
for path in paths:
  if 'result' in paths[path]:
    create_directory(paths[path])

# 2. Preprocessing
Creates spectrograms in HDF5 format and splits as txt files.

## Preprocessing from Samek et al.

In [ ]:
# the following code is the preprocessing from https://github.com/soerenab/AudioMNIST/blob/master/preprocess_data.py
# I did not write this whole code snippet
# adjustments were made, see next cell
def preprocess_data(src, dst, src_meta, n_processes=15):

    """
    Calls for distibuted preprocessing of the data.
    Parameters:
    -----------
        src: string
            Path to data directory.
        dst: string
            Path to directory where preprocessed data shall be stored.
        stc_meta: string
            Path to meta_information file.
        n_processes: int
            number of simultaneous processes to use for data preprocessing.
    """

    folders = []

    for folder in os.listdir(src):
        # only process folders
        if not os.path.isdir(os.path.join(src, folder)):
            continue
        folders.append(folder)

    pool=multiprocessing.Pool(processes=n_processes)
    _=pool.map(_preprocess_data, [(os.path.join(src, folder), 
                                          os.path.join(dst, folder), 
                                          src_meta) for folder in sorted(folders)])

def _preprocess_data(src_dst_meta):

    """
    Preprocessing for all data files in given directory.
    Preprocessing includes:
        AlexNet: resampling to 8000 Hz, 
                 embedding in zero vector, 
                 transformation to amplitute spectrogram representation in dB.
        
        AudioNet: resampling to 8000 Hz, 
                  embedding in zero vector, 
                  normalization at 95th percentile.
    Preprocessed data will be stored in hdf5 files with one datum per file.
    In terms of I/O, this is not very efficient but it allows to easily change
    training, validation, and test sets without re-preprocessing or redundant 
    storage of preprocessed files.
    Parameters:
    -----------
        src_dst_meta: tuple of 3 strings
            Tuple (path to data directory, path to destination directory, path
            to meta file)
    """


    src, dst, src_meta = src_dst_meta

    print("processing {}".format(src))

    metaData = json.load(open(src_meta))

    # create folder for hdf5 files
    if not os.path.exists(dst):
        os.makedirs(dst)
    # loop over recordings
    for filepath in sorted(glob.glob(os.path.join(src, "*.wav"))):

        # infer sample info from name
        dig, vp, rep = filepath.rstrip(".wav").split("/")[-1].split("_")
        # read data
        fs, data = wavf.read(filepath)
        # resample
        data = librosa.core.resample(y=data.astype(np.float32), orig_sr=fs, target_sr=8000, res_type="scipy")
        # zero padding
        if len(data) > 8000:
            raise ValueError("data length cannot exceed padding length.")
        elif len(data) < 8000:
            embedded_data = np.zeros(8000)
            offset = np.random.randint(low = 0, high = 8000 - len(data))
            embedded_data[offset:offset+len(data)] = data
        elif len(data) == 8000:
            # nothing to do here
            embedded_data = data
            pass

        ##### AlexNet #####

        # stft, with seleced parameters, spectrogram will have shape (228,230)
        f, t, Zxx = scipy.signal.stft(embedded_data, 8000, nperseg = 455, noverlap = 420, window='hann')
        # get amplitude
        Zxx = np.abs(Zxx[0:227, 2:-1])
        Zxx = np.atleast_3d(Zxx).transpose(2,0,1)
        # convert to decibel
        Zxx = librosa.amplitude_to_db(Zxx, ref = np.max)
        # save as hdf5 file
        with h5py.File(os.path.join(dst, "AlexNet_{}_{}_{}.hdf5".format(dig, vp, rep)), "w") as f:
            tmp_X = np.zeros([1, 1, 227, 227])

            tmp_X[0, 0] = Zxx
            f['data'] = tmp_X
            f['label'] = np.array([[int(dig), 0 if metaData[vp]["gender"] == "male" else 1]])

        ##### AudioNet #####
        
        embedded_data /= (np.percentile(embedded_data, 95) + 0.001)
        
        with h5py.File(os.path.join(dst, "AudioNet_{}_{}_{}.hdf5".format(dig, vp, rep)), "w") as f:
            tmp_X = np.zeros([1, 1, 1, 8000])

            tmp_X[0, 0, 0] = embedded_data
            f['data'] = tmp_X
            f['label'] = np.array([[int(dig), 0 if metaData[vp]["gender"] == "male" else 1]])

    return

def create_splits(src, dst):

    """
    Creation of text files specifying which files training, validation and test
    set consist of for each cross-validation split. 
    Parameters:
    -----------
        src: string
            Path to directory containing the directories for each subject that
            hold the preprocessed data in hdf5 format.
        dst: string
            Destination where to store the text files specifying training, 
            validation and test splits.
    """

    #print("creating splits")
    splits={"digit":{   "train":[   set([28, 56,  7, 19, 35,  1,  6, 16, 23, 34, 46, 53, 36, 57,  9, 24, 37,  2, \
                                          8, 17, 29, 39, 48, 54, 43, 58, 14, 25, 38,  3, 10, 20, 30, 40, 49, 55]),

                                    set([36, 57,  9, 24, 37,  2,  8, 17, 29, 39, 48, 54, 43, 58, 14, 25, 38,  3, \
                                         10, 20, 30, 40, 49, 55, 12, 47, 59, 15, 27, 41,  4, 11, 21, 31, 44, 50]),

                                    set([43, 58, 14, 25, 38,  3, 10, 20, 30, 40, 49, 55, 12, 47, 59, 15, 27, 41, \
                                          4, 11, 21, 31, 44, 50, 26, 52, 60, 18, 32, 42,  5, 13, 22, 33, 45, 51]),

                                    set([12, 47, 59, 15, 27, 41,  4, 11, 21, 31, 44, 50, 26, 52, 60, 18, 32, 42, \
                                          5, 13, 22, 33, 45, 51, 28, 56,  7, 19, 35,  1,  6, 16, 23, 34, 46, 53]),

                                    set([26, 52, 60, 18, 32, 42,  5, 13, 22, 33, 45, 51, 28, 56,  7, 19, 35,  1, \
                                          6, 16, 23, 34, 46, 53, 36, 57,  9, 24, 37,  2,  8, 17, 29, 39, 48, 54])],

                        "validate":[set([12, 47, 59, 15, 27, 41,  4, 11, 21, 31, 44, 50]),
                                    set([26, 52, 60, 18, 32, 42,  5, 13, 22, 33, 45, 51]),
                                    set([28, 56,  7, 19, 35,  1,  6, 16, 23, 34, 46, 53]),
                                    set([36, 57,  9, 24, 37,  2,  8, 17, 29, 39, 48, 54]),
                                    set([43, 58, 14, 25, 38,  3, 10, 20, 30, 40, 49, 55])],

                        "test":[    set([26, 52, 60, 18, 32, 42,  5, 13, 22, 33, 45, 51]),
                                    set([28, 56,  7, 19, 35,  1,  6, 16, 23, 34, 46, 53]),
                                    set([36, 57,  9, 24, 37,  2,  8, 17, 29, 39, 48, 54]),
                                    set([43, 58, 14, 25, 38,  3, 10, 20, 30, 40, 49, 55]),
                                    set([12, 47, 59, 15, 27, 41,  4, 11, 21, 31, 44, 50])]},

            "gender":{  "train":[   set([36, 47, 56, 26, 12, 57, 2, 44, 50, 25, 37, 45]),
                                    set([26, 12, 57, 43, 28, 52, 25, 37, 45, 48, 53, 41]),
                                    set([43, 28, 52, 58, 59, 60, 48, 53, 41, 7, 23, 38]),
                                    set([58, 59, 60, 36, 47, 56, 7, 23, 38, 2, 44, 50])],

                        "validate":[set([43, 28, 52, 48, 53, 41]),
                                    set([58, 59, 60, 7, 23, 38]),
                                    set([36, 47, 56, 2, 44, 50]),
                                    set([26, 12, 57, 25, 37, 45])],

                        "test":[    set([58, 59, 60, 7, 23, 38]),
                                    set([36, 47, 56, 2, 44, 50]),
                                    set([26, 12, 57, 25, 37, 45]),
                                    set([43, 28, 52, 48, 53, 41])]}}

    for split in range(5):
        for modus in ["train", "validate", "test"]:
            for task in ["digit", "gender"]:
                if task == "gender" and split > 3:
                    continue
                with open(os.path.join(dst, "AlexNet_{}_{}_{}.txt".format(task, split, modus)), mode = "w") as txt_file:
                    for vp in splits[task][modus][split]:
                        for filepath in glob.glob(os.path.join(src, "{:02d}".format(vp), "AlexNet*.hdf5")):
                            txt_file.write(filepath+"\n")

                with open(os.path.join(dst, "AudioNet_{}_{}_{}.txt".format(task, split, modus)), mode = "w") as txt_file:
                    for vp in splits[task][modus][split]:
                        for filepath in glob.glob(os.path.join(src, "{:02d}".format(vp), "AudioNet*.hdf5")):
                            txt_file.write(filepath+"\n")


######################################################## adjustment starting here #######################################################################################################

# if __name__ == "__main__":

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--source', '-src', default=os.path.join(os.getcwd(), "data"), help="Path to folder containing each participant's data directory.")
#     parser.add_argument('--destination', '-dst', default=os.path.join(os.getcwd(), "preprocessed_data"), help="Destination where preprocessed data shall be stored.")
#     parser.add_argument('--meta', '-m', default=os.path.join(os.getcwd(), "data", "audioMNIST_meta.txt"), help="Path to meta_information json file.")

#     args = parser.parse_args()

#     # preprocessing
#     preprocess_data(src=args.source, dst=args.destination, src_meta=args.meta)
#     # create training, validation and test sets
#     create_splits(src=args.destination, dst=args.destination)

## Adjustments and extensions for the preprocessing

In [ ]:
# download dataset AudioMNIST
if os.path.isdir(paths['dataset']) == False:
  try:
    start_time = time.time()
    url = 'https://github.com/soerenab/AudioMNIST/archive/refs/heads/master.zip'
    write_log('Downloading AudioMNIST dataset ...')
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    write_log('Unzipping AudioMNIST dataset ...')
    zipfile.extractall(path=paths['root'])
    write_log('Download sucessful, dataset is ready now! (download time: '+get_duration(start_time,time.time()))
  except:
    write_log('AudioMNIST download failed!')
else:
  write_log('Data already exists, no download necessary!')

# # paths for preprocessing
# data_dir = os.path.join(dataset_dir,'data')
# metafile_path = os.path.join(dataset_dir,'data','audioMNIST_meta.txt')
# spectrogram_dir = os.path.join(root_dir,'spectrograms')
# split_dir = os.path.join(root_dir,'splits')

# start preprocessing

# create spectrograms: hdf5 file with spectrogram data and labels (digit and gender) for each wav-file
if os.path.isdir(paths['spectrograms']) == False:
  start_time = time.time()
  write_log('Creating spectrograms ...')
  preprocess_data(src=paths['data'], dst=paths['spectrograms'], src_meta=paths['meta'])
  write_log('Spectrograms created in '+get_duration(start_time,time.time()))
else:
  write_log('Spectrograms already exist, no preprocessing necessary!')

# creat splits
if os.path.isdir(paths['splits']) == False:
  create_directory(paths['splits'])
  start_time = time.time()
  # create training, validation and test sets
  write_log('Creating splits ...')
  create_splits(src=paths['spectrograms'], dst=paths['splits'])
  write_log('Splits created in '+get_duration(start_time,time.time()))
else:
  write_log('Splits already exist, no preprocessing necessary!')

Data already exists, no download necessary!
Spectrograms already exist, no preprocessing necessary!
Splits already exist, no preprocessing necessary!


# 3. Train models

## General network settings

In [ ]:
# Model configuration
batch_size = 32 # spectrograms per step
img_width, img_height, img_num_channels = 227, 227, 1 # image size and channels (3=RGB, 1=Grayscale)
input_shape = (img_width, img_height, img_num_channels) # input shape for the first layer
loss_function = sparse_categorical_crossentropy # loss function
no_epochs = 100 # rounds of training
optimizer = tf.optimizers.SGD(lr=0.001)  # optimizer Adam(amsgrad=True)
verbosity = 1
resize_factor = 256 #256

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Generate model architecture

In [ ]:
def get_model_architecture(input_shape,no_classes,activation):
  model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=input_shape),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'), #4096
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024, activation='relu'), #4096
    keras.layers.Dropout(0.5),
    keras.layers.Dense(no_classes, activation=activation)
  ])
  print(model.summary())
  return model

## start model trainings

In [ ]:
write_log('Started training of the models!')
# iterate labels and splits
for label in labels:
  for split in range(0,splits):
# for label in ['gender']: #replace, its only for testing
#   for split in range(1,2): # replace, its only for testing
    if label == "gender" and split > 3: # there are only 4 splits for the label gender, whereas there are 5 for label digit
      continue
    write_log('Current label: '+str(label)+' and current split: '+str(split))
    #check if the model already exists
    model_saving_path = os.path.join(paths['models'],'AlexNet_'+label+'_'+str(split)+'.h5')
    history_saving_path = os.path.join(paths['history'],'AlexNet_'+label+'_'+str(split)+'.pkl')
    if os.path.isfile(model_saving_path) == False:
      # call functions to read data for train and validation
      x_train, y_train = read_data(label,split,'train')
      x_val, y_val = read_data(label,split,'validate')
      # resize x
      x_train = x_train/resize_factor
      x_val = x_val/resize_factor
      # Reshape data, must be (no_spectrograms, 227,227,1) instead of (no_spectrograms, 227,227)
      x_train = x_train.reshape((len(x_train), img_width, img_height, img_num_channels))
      x_val  = x_val.reshape((len(x_val), img_width, img_height, img_num_channels))
      # set number of classes
      if label == 'gender':
        no_classes = 2
        activation = 'sigmoid'
      else: 
        no_classes = 10
        activation = 'softmax'
      # get model structure
      model = get_model_architecture(input_shape,no_classes,activation)
      # Compile the model
      model.compile(loss=loss_function, #loss='binary_crossentropy'
              optimizer=optimizer,
              metrics=['accuracy'])
      # train current model
      write_log('Training the model ...')
      start_time = time.time()
      history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_data = (x_val, y_val))
      write_log('Training finished in '+get_duration(start_time,time.time()))
      # save current model and history
      write_log('Saving the model and history ...')
      pickle.dump(history.history, open(history_saving_path, 'wb'))
      model.save(model_saving_path)
      write_log('Model and history saved!')
    else:
      write_log('Model already exists')

Started training of the models!
Current label: gender and current split: 0
Model already exists
Current label: gender and current split: 1
Model already exists
Current label: gender and current split: 2
Model already exists
Current label: gender and current split: 3
Model already exists
Current label: digit and current split: 0
Model already exists
Current label: digit and current split: 1
Model already exists
Current label: digit and current split: 2
Model already exists
Current label: digit and current split: 3
Model already exists
Current label: digit and current split: 4
Model already exists


# 4. Evaluate models

## Evaluate all models

In [ ]:
# free some RAM if training was conducted before
x_train,y_train,x_val,y_val = 0,0,0,0
# create list of trained models
write_log('Model evaluation started ...')
model_names = os.listdir(paths['models'])
# iterate models
for model_name in model_names:
  # get label and split from modelname
  net,label,split_index = model_name.rstrip(".h5").split("/")[-1].split("_")
  # set saving paths for the results
  evaluation_path = os.path.join(paths['evaluation'],'evaluation_'+label+'_'+str(split_index)+'.csv')
  confusion_matrix_path = os.path.join(paths['evaluation'],'confusionMatrix_'+label+'_'+str(split_index)+'.csv')
  # check if model was already evaluated
  if os.path.isfile(evaluation_path) and os.path.isfile(confusion_matrix_path):
    write_log('Model for '+label+' '+str(split_index)+' already evaluated')
  else:
    write_log('Current model: '+label+' '+str(split_index))
    # load current model
    cur_model = load_model(os.path.join(paths['models'],model_name))
    # read according test data
    x_test, y_test = read_data(label,split_index,'test')
    # Reshape data, must be (no_spectrograms, 227,227,1) instead of (no_spectrograms, 227,227)
    x_test = x_test.reshape((len(x_test), img_width, img_height, img_num_channels))
    # Run the Class Predictions and get the probabilities
    start_time = time.time()
    pred_probas = cur_model.predict(x_test, verbose=1)
    predictions = np.argmax(pred_probas,axis=1)
    # calculate evaluation metrics
    if os.path.isfile(evaluation_path) == False:
      evaluation = pd.DataFrame(data=np.zeros((1,7),dtype=np.float), columns=['label','split','accuracy','balanced accuracy','precision','recall','kappa'])
      evaluation['label'] = label
      evaluation['split'] = split_index
      evaluation['accuracy'] = accuracy_score(y_test, predictions)
      evaluation['balanced accuracy'] = balanced_accuracy_score(y_test, predictions)
      evaluation['precision'] = precision_score(y_test, predictions, average='weighted')
      evaluation['recall'] = recall_score(y_test, predictions, average='weighted')
      evaluation['kappa'] = cohen_kappa_score(y_test, predictions)
      #roc_AUC = roc_auc_score(y_test, pred_probas) # not working for multiclass?!
      # calculate accuracy and test loss
      # score = cur_model.evaluate(x_test, y_test, verbose=0)
      # test_loss = score[0]
      # accuracy = score[1]
      evaluation.to_csv(evaluation_path, index=False)
    # calculate confusion matrix
    if os.path.isfile(confusion_matrix_path) == False:
      confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Actual'], colnames=['Predicted'])
      confusion_matrix.to_csv(confusion_matrix_path, index= False)
    write_log('Evaluation of model '+label+' '+str(split_index)+' finished in '+str(get_duration(start_time,time.time())))
write_log('All evaluations finished!')

Model evaluation started ...
Model for gender 0 already evaluated
Model for gender 1 already evaluated
Model for gender 2 already evaluated
Model for gender 3 already evaluated
Model for digit 0 already evaluated
Model for digit 1 already evaluated
Model for digit 2 already evaluated
Model for digit 3 already evaluated
Model for digit 4 already evaluated
All evaluations finished!


## Calculate mean results

In [ ]:
write_log('Calculating mean results ...')
start_time = time.time()
# create variables to store the results
gender_evaluation_data = pd.DataFrame(data=np.zeros((0,7),dtype=np.float), columns=['label','split','accuracy','balanced accuracy','precision','recall','kappa'])
gender_confusion_matrix_data = pd.DataFrame(data=np.zeros((0,0),dtype=np.float))
digit_evaluation_data = pd.DataFrame(data=np.zeros((0,7),dtype=np.float), columns=['label','split','accuracy','balanced accuracy','precision','recall','kappa'])
digit_confusion_matrix_data = pd.DataFrame(data=np.zeros((0,0),dtype=np.float))
# define counters
gender_confusion_matrix_counter,digit_confusion_matrix_counter = 0,0
# iterate all available result files
for cur_evaluation_name in os.listdir(paths['evaluation']):
  # exclude possibly existing mean files
  if 'mean' in cur_evaluation_name:
    continue
  # get evaluation type, label and split from saved result csv
  evaluation_type,label,split_index = cur_evaluation_name.rstrip(".csv").split("/")[-1].split("_")
  # read results of the models
  cur_data = pd.read_csv(os.path.join(paths['evaluation'],cur_evaluation_name))
  # add current results to the correct data collection
  if evaluation_type == 'evaluation':
    if label == 'gender':
      gender_evaluation_data = pd.concat((gender_evaluation_data,cur_data),axis=0,sort=False)
    else:
      digit_evaluation_data = pd.concat((digit_evaluation_data,cur_data),axis=0,sort=False)
  else:
    if label == 'gender':
      gender_confusion_matrix_data = gender_confusion_matrix_data.add(cur_data, fill_value=0)
      gender_confusion_matrix_counter += 1
    else:
      digit_confusion_matrix_data = digit_confusion_matrix_data.add(cur_data, fill_value=0)
      digit_confusion_matrix_counter += 1
# calculate mean values
gender_evaluation_data.loc['mean'] = gender_evaluation_data.mean()
gender_evaluation_data.at['mean', 'label'] ='gender'
gender_evaluation_data.at['mean', 'split'] ='mean'
digit_evaluation_data.loc['mean'] = digit_evaluation_data.mean()
digit_evaluation_data.at['mean', 'label'] ='digit'
digit_evaluation_data.at['mean', 'split'] ='mean'
gender_confusion_matrix_mean = gender_confusion_matrix_data/gender_confusion_matrix_counter
digit_confusion_matrix_mean = digit_confusion_matrix_data/digit_confusion_matrix_counter
# save mean results to csv
gender_evaluation_data.to_csv(os.path.join(paths['evaluation'],'evaluation_gender_mean.csv'),index = False)
digit_evaluation_data.to_csv(os.path.join(paths['evaluation'],'evaluation_digit_mean.csv'),index = False)
gender_confusion_matrix_mean.to_csv(os.path.join(paths['evaluation'],'confusionMatrix_gender_mean.csv'),index = False)
digit_confusion_matrix_mean.to_csv(os.path.join(paths['evaluation'],'confusionMatrix_digit_mean.csv'),index = False)
write_log('Mean results calculated in '+get_duration(start_time,time.time()))

Calculating mean results ...
Mean results calculated in 00:00:06


In [ ]:
# # not working for multiclass
# fprT, tprT, _ = roc_curve(y_test, predictions)

# plt.figure(figsize=(9,7))

# plt.plot(fprT, tprT, label='ROC Test Split (AUC: %.2f)' %  roc_auc_score(y_test, predictions))
# plt.plot([0, 1], [0, 1], linestyle='--', color='k')
# plt.xlim([-0.03, 1.03])
# plt.ylim([-0.03, 1.03])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve: SimpleCNN Cats vs Dogs CNN')
# plt.legend(loc="lower right", facecolor='white', edgecolor='white')
# plt.show()

In [ ]:
# history_saving_path = os.path.join(paths['history'],'AlexNet_gender_0.pkl')
# history = pickle.load(open(history_saving_path,'rb'))

# plt.figure(figsize=(9,7))
# plt.plot(history['loss'])
# plt.plot(history['val_loss'])
# plt.title('Train loss')
# plt.ylabel('Model Loss')
# plt.xlabel('Epoch')
# plt.legend(['train', 'validate'], loc='center right')
# plt.show()

# 5. Run XAI methods

## Gradient-weighted Class Activation Mapping (Grad-CAM)

In [ ]:
xai_methods = ['Grad-CAM','LIME','LRP','DTD','OCC']
# create list of trained models
write_log('XAI methods started ...')
model_names = os.listdir(paths['models'])
# iterate models
for model_name in model_names:
  # load current model
  cur_model = load_model(os.path.join(paths['models'],model_name))
  # get label and split from modelname
  net,label,split_index = model_name.rstrip(".h5").split("/")[-1].split("_")




#   # set saving paths for the results
#   evaluation_path = os.path.join(paths['evaluation'],'evaluation_'+label+'_'+str(split_index)+'.csv')
#   confusion_matrix_path = os.path.join(paths['evaluation'],'confusionMatrix_'+label+'_'+str(split_index)+'.csv')
#   # check if model was already evaluated
#   if os.path.isfile(evaluation_path) and os.path.isfile(confusion_matrix_path):
#     write_log('Model for '+label+' '+str(split_index)+' already evaluated')
#   else:
#     write_log('Current model: '+label+' '+str(split_index))
#     # read according test data
#     x_test, y_test = read_data(label,split_index,'test')
#     # Reshape data, must be (no_spectrograms, 227,227,1) instead of (no_spectrograms, 227,227)
#     x_test = x_test.reshape((len(x_test), img_width, img_height, img_num_channels))
#     # Run the Class Predictions and get the probabilities
#     start_time = time.time()
#     pred_probas = cur_model.predict(x_test, verbose=1)
#     predictions = np.argmax(pred_probas,axis=1)
#     # calculate evaluation metrics
#     if os.path.isfile(evaluation_path) == False:
#       evaluation = pd.DataFrame(data=np.zeros((1,7),dtype=np.float), columns=['label','split','accuracy','balanced accuracy','precision','recall','kappa'])
#       evaluation['label'] = label
#       evaluation['split'] = split_index
#       evaluation['accuracy'] = accuracy_score(y_test, predictions)
#       evaluation['balanced accuracy'] = balanced_accuracy_score(y_test, predictions)
#       evaluation['precision'] = precision_score(y_test, predictions, average='weighted')
#       evaluation['recall'] = recall_score(y_test, predictions, average='weighted')
#       evaluation['kappa'] = cohen_kappa_score(y_test, predictions)
#       #roc_AUC = roc_auc_score(y_test, pred_probas) # not working for multiclass?!
#       # calculate accuracy and test loss
#       # score = cur_model.evaluate(x_test, y_test, verbose=0)
#       # test_loss = score[0]
#       # accuracy = score[1]
#       evaluation.to_csv(evaluation_path, index=False)
#     # calculate confusion matrix
#     if os.path.isfile(confusion_matrix_path) == False:
#       confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Actual'], colnames=['Predicted'])
#       confusion_matrix.to_csv(confusion_matrix_path, index= False)
#     write_log('Evaluation of model '+label+' '+str(split_index)+' finished in '+str(get_duration(start_time,time.time())))
# write_log('All evaluations finished!')

In [ ]:
from gradcam import VizGradCAM

In [ ]:
#test_img = img_to_array(load_img("monkey.jpeg" , target_size=(224,224)))
test_model = load_model(os.path.join(paths['models'],'AlexNet_gender_0.h5'))
test_img = /gdrive/My Drive/Xplainable AI - Univariate Time Series - spectrograms/spectrograms/38/AlexNet_0_38_0.hdf5

# Use The Function - Boom!
VizGradCAM(test_model, test_img))

SyntaxError: ignored

In [ ]:
#test_model = load_model(os.path.join(paths['models'],'AlexNet_gender_0.h5'))
cur_path = '/gdrive/My Drive/Xplainable AI - Univariate Time Series - spectrograms/spectrograms/38/AlexNet_0_38_0.hdf5'
f = h5py.File(cur_path, 'r')
x_cur = f['data'][...]
y_cur = f['label'][...]
f.close()

x_cur = x_cur[0][0]
y_cur = y_cur[0][0] 

# Reshape data, must be (no_spectrograms, 227,227,1) instead of (no_spectrograms, 227,227)
x_cur = x_cur.reshape(1, 227,227,1)

x_cur.shape
VizGradCAM(test_model, x_cur)

# index = 0
#   x = np.zeros(((len(split_paths),227,227))) # create target array for spectrograms
#   y = np.zeros(len(split_paths)) # create target array for labels
#   for cur_path in split_paths: # iterate the files
#     #read current file
#     f = h5py.File(cur_path, 'r')
#     x_cur = f['data'][...]
#     y_cur = f['label'][...]
#     f.close() 
#     #extract relevant data of current file
#     x_cur = x_cur[0][0]
#     y_cur = y_cur[0][label_index]    
#     #append current data to x and y
#     x[index] = x_cur
#     y[index] = y_cur
#     # increase index by 1
#     index +=1

## Local interpretable model-agnostic explanations (LIME)

## Layerwise Relevance Propagation (LRP)

## Deep Taylor Decomposition (DTD)

## Occlusion Analysis (OCC)

# Additional inspections

## Create plots of waveform/spectrogram data

### functions for single files

In [ ]:
# function creates a waveform plot for a single .wav file
# src is the path to the .wav file
# dst is the path were the plot is saved to
# optional the y-axis can be scaled between -1 and 1 (default True)
# optional the plot can be shown (default False)
def create_waveform_plot(src,dst,scale=True,show=False):
  if os.path.isfile(dst):
    return
  create_directory(os.path.join(paths['plots_waveform'],dst.split('/')[-2])) #create the subfolder with participant number, eg. 01
  fs, data = wavf.read(src)
  if scale:
    data = minmax_scale(data,feature_range=(-1,1),axis=0,copy=True) # optional: scales the y-axis between -1 and 1
  duration = len(data)/fs #duration of the file
  time = np.arange(0,duration,1/fs) #time vector
  # sometimes time and data dont have the same length which causes an error: fix by adjusting the length
  if len(time) != len(data):
    if len(time)>len(data):
      time = time[0:len(data)]
    else:
      data = data[0:len(time)]
  plt.plot(time,data)
  plt.xlabel('Time [s]')
  plt.ylabel('Amplitude')
  plt.savefig(dst)
  if show:
    plt.show()
  plt.close()
  return

# function creates a spectrogram plot for a single .hdf5 file - analog to the function for waveform files
def create_spectrogram_plot(src,dst,show=False):
  if os.path.isfile(dst):
    return
  create_directory(os.path.join(paths['plots_spectrograms'],dst.split('/')[-2])) #create the subfolder with participant number, eg. 01
  #read current file
  f = h5py.File(src, 'r')
  data = f['data'][...]
  f.close() 
  data = np.squeeze(data) # alternatively use: data = data[0,0]
  plt.pcolormesh(data, cmap='Greys')
  plt.axis('off')
  #plt.xlabel('Time [s]')
  #plt.ylabel('Hz')
  plt.savefig(dst)
  if show:
    plt.show()
  plt.close()
  return

### functions to iterate all files or some examples

In [ ]:
# if you want to create the waveform plots for all .wav files or some examples for each participant, the following function iterates the files
def create_plots(src, dst, src_meta, input_type, n_processes=5, mode = 'examples', nb_examples = 3,scale=True):
  if mode == 'examples':
    write_log('Started creating plots of '+input_type+' for '+str(nb_examples)+' examples for each digit of each participant ...')
  elif mode == 'all':
    write_log('Started creating plots of '+input_type+' for all files ...')
  else:
    write_log('No valid mode!')
  start_time = time.time()
  folders = []
  for folder in os.listdir(src):
    # only process folders
    if not os.path.isdir(os.path.join(src, folder)):
      continue
    folders.append(folder)
  # start multiprocessing to speed up the creation of the png files
  pool=multiprocessing.Pool(processes=n_processes)
  _=pool.map(_create_plots, [(os.path.join(src, folder), 
                                          os.path.join(dst, folder), 
                                          src_meta,input_type,mode,nb_examples,scale) for folder in sorted(folders)])
  write_log('Finished creating plots in '+str(get_duration(start_time,time.time())))

def _create_plots(parameters):
  src, dst, src_meta, input_type, mode, nb_examples,scale = parameters
  #print("processing {}".format(src))
  metaData = json.load(open(src_meta))
  # create folder for png files
  if not os.path.exists(dst):
    os.makedirs(dst)
  # check input type and get corresponding file format
  if input_type == 'waveform':
    file_format = '.wav'
  elif input_type == 'spectrogram':
    file_format = '.hdf5'
  else:
    write_log('No valid input type!')
    return
  # define counter
  counter = [0,0,0,0,0,0,0,0,0,0]   
  # loop over recordings
  for filepath in sorted(glob.glob(os.path.join(src, '*'+file_format))):
    # distinguish between waveform and spectrograms   
    if input_type == 'waveform':     
      dig, vp, rep = filepath.rstrip(file_format).split("/")[-1].split("_") # infer sample info from name
      if counter[int(dig)] == int(nb_examples): # continue if enough examples are created for the current digit
        continue
      gender = 0 if metaData[vp]["gender"] == "male" else 1 # get gender
      saving_path = os.path.join(dst,str(gender)+'_'+str(dig)+'_'+str(vp)+'_'+str(rep)+'.png') # define saving path
      create_waveform_plot(filepath,saving_path, show=False, scale=scale)
    else:
      net, dig, vp, rep = filepath.rstrip(".hdf5").split("/")[-1].split("_") # infer sample info from name
      if counter[int(dig)] == int(nb_examples): # continue if enough examples are created for the current digit
        continue
      gender = 0 if metaData[vp]["gender"] == "male" else 1 # get gender
      saving_path = os.path.join(dst,net+'_'+str(gender)+'_'+str(dig)+'_'+str(vp)+'_'+str(rep)+'.png')# define saving path
      create_spectrogram_plot(filepath,saving_path,show=False) 
    # check if all files should be iterated, if not: increase counter by 1
    if mode == 'all':
      continue
    else:
      counter[int(dig)] +=1     
  return 

### Select and start functions

In [ ]:
# create one waveform plot
#create_waveform_plot(os.path.join(paths['data'],'56/5_56_0.wav'),os.path.join(paths['plots_waveform'],'56/1_5_56_0.png'),show=True)
# create all or example plots for wavelet files
create_plots(src=paths['data'],dst=paths['plots_waveform'],src_meta=paths['meta'],mode='examples',input_type='waveform',nb_examples=3,scale=True, n_processes=5)
# create one spectrogram plot
#create_spectrogram_plot(os.path.join(paths['spectrograms'],'56/AlexNet_5_56_0.hdf5'),os.path.join(paths['plots_spectrograms'],'56/AlexNet_1_5_56_0.png'),show=True)
# create all or example plots for spectrogram files
create_plots(src=paths['spectrograms'],dst=paths['plots_spectrograms'],src_meta=paths['meta'],mode='examples',input_type='spectrogram',nb_examples=3, n_processes=5)

Started creating plots of spectrogram for 3 examples for each digit of each participant ...
Finished creating plots in 00:00:52
